In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

classifier = pipeline("ner", model="stulcrad/CNEC_2_0_robeczech-base")
str_ = "Jste světa znalý muž a víte stejně dobře jako já , že souvislost mezi současnými krutostmi v Jihovýchodní Asii a tou novou bankovní pobočkou hned vedle obchoďáku Zátoka je přímá a bezprostřední ; byl z toho už vzteklý jak uvázaný pes , protože zájemci o hodiny mu úplně narušili jeho denní režim a on si nemohl po obědě ani zdřímnout ."
classifier(str_)

In [ ]:
str_[162:168]

In [ ]:
import json
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

def combine_adjacent_name_tokens(results):
    """
    Spojí navazující tokeny jmen do celých jmen.
    
    Args:
        results: Seznam tokenů z NER modelu.
    
    Returns:
        Seznam slovníků obsahujících spojená jména.
    """
    # Typy tokenů pro jména
    person_types = ['pf', 'ps', 'pp', 'pm', 'pc', 'pd' ]
    
    combined_names = []
    current_name = None
    last_index = -1
    
    # Seřadit výsledky podle indexu
    sorted_results = sorted(results, key=lambda x: x['index'])
    
    for token in sorted_results:
        # Kontrola, zda je token část jména
        entity_type = token['entity'].split('-')[1] if '-' in token['entity'] else None
        is_name_part = entity_type in person_types
        
        if is_name_part:
            if current_name is None or token['index'] != last_index + 1:
                # Začít nové jméno, pokud nejde o následující token
                if current_name:
                    combined_names.append(current_name)
                current_name = {
                    'text': token['word'].lstrip('Ġ'),  # odstranit prefix tokenizéru
                    'start': token['start'],
                    'end': token['end'],
                    'tokens': [token]
                }
            else:
                # Přidat k současnému jménu - speciální zpracování pro interpunkci
                if token['word'] == 'Ġ.':
                    current_name['text'] += '.'  # bez mezery pro tečku
                else:
                    # Pro ostatní tokeny
                    current_name['text'] += token['word'].lstrip('Ġ')
                current_name['end'] = token['end']
                current_name['tokens'].append(token)
            
            last_index = token['index']
    
    # Přidat poslední jméno
    if current_name:
        combined_names.append(current_name)
        
    return combined_names

def extract_and_mark_names(text, results):
    """
    Ohraničí nalezená jména značkami @@ a ##.
    
    Args:
        text: Původní text.
        results: Výstupy z NER modelu.
    
    Returns:
        Text s označenými jmény.
    """
    # Získáme pozice všech jmen
    combined = combine_adjacent_name_tokens(results)
    
    # Seřadíme podle začátku
    combined.sort(key=lambda x: x['start'])
    
    # Vytvoříme nový text s označenými jmény
    marked_text = ""
    last_end = 0
    
    for name in combined:
        # Přidáme text před jménem
        marked_text += text[last_end:name['start']]
        
        # Přidáme označené jméno
        original_name = text[name['start']:name['end']]
        marked_text += "@@" + original_name + "##"
        
        # Aktualizujeme pozici
        last_end = name['end']
    
    # Přidáme zbývající text po posledním jménu
    marked_text += text[last_end:]
    
    return marked_text

def process_dataset_with_model(input_file, output_file):
    """
    Zpracuje dataset pomocí modelu a uloží označené kontexty do výstupního souboru.
    
    Args:
        input_file: Cesta k vstupnímu JSON souboru.
        output_file: Cesta k výstupnímu textovému souboru.
    """
    # Načtení modelu
    model = AutoModelForTokenClassification.from_pretrained("stulcrad/CNEC_2_0_robeczech-base")
    tokenizer = AutoTokenizer.from_pretrained("stulcrad/CNEC_2_0_robeczech-base")
    classifier = pipeline("ner", model=model, tokenizer=tokenizer)
    
    # Načtení datasetu
    with open(input_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # Zpracování a uložení výsledků
    with open(output_file, 'w', encoding='utf-8') as f:
        for item in data:
            context = item["context"]
            results = classifier(context)
            marked_context = extract_and_mark_names(context, results)
            f.write(marked_context + '\n')

# Hlavní funkce
if __name__ == "__main__":
    input_file = "input_dataset.json"
    output_file = "hokuspokus.txt"
    process_dataset_with_model(input_file, output_file)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [6]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
str = "Zasáhli u pořadu z Františka Halase Naše paní Božena Němcová , což bylo v dobách prehistorie souboru ."
model = AutoModelForTokenClassification.from_pretrained("stulcrad/CNEC_2_0_robeczech-base")
tokenizer = AutoTokenizer.from_pretrained("stulcrad/CNEC_2_0_robeczech-base")
classifier = pipeline("ner", model=model, tokenizer=tokenizer)
results = classifier(str)
print(results)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity': 'B-pf', 'score': 0.9994073, 'index': 7, 'word': 'ĠFrantiÅ¡ka', 'start': 19, 'end': 28}, {'entity': 'B-ps', 'score': 0.9985337, 'index': 8, 'word': 'ĠHala', 'start': 29, 'end': 33}, {'entity': 'I-ps', 'score': 0.99907947, 'index': 9, 'word': 'se', 'start': 33, 'end': 35}, {'entity': 'B-oa', 'score': 0.99799675, 'index': 10, 'word': 'ĠNaÅ¡e', 'start': 36, 'end': 40}, {'entity': 'I-oa', 'score': 0.9988361, 'index': 11, 'word': 'ĠpanÃŃ', 'start': 41, 'end': 45}, {'entity': 'I-oa', 'score': 0.9988016, 'index': 12, 'word': 'ĠBoÅ¾ena', 'start': 46, 'end': 52}, {'entity': 'I-oa', 'score': 0.9984163, 'index': 13, 'word': 'ĠNÄĽmcovÃ¡', 'start': 53, 'end': 60}]
